# Notes

* Streamlit ile bir uygulamayı localde çalıştırmak, modelin deploy edildiği anlamına gelmez.
Modelin gerçekten deploy edilmiş sayılması için, bir sunucuda 7/24 aktif şekilde çalışıyor olması ve bu sunucuya gelen istekleri (request) işleyip yanıt (response) verebilmesi gerekir.

* Normalde doğrudan main branch'e push atılmaz. Bunun için main branch'i readonly yapabileceğimiz bir ruleset tanımlayabiliriz.

* Projeyi açtıktan sonra, öncelikle projenin ihtiyaç duyduğu tüm kütüphane ve paketlerin doğru şekilde yönetilebilmesi için bir dependency management (bağımlılık yönetimi) sistemi kurmamız gerekir.
Bu, projenin hangi kütüphanelere hangi sürümlerde ihtiyaç duyduğunu belirlemek ve bu ortamı izole bir şekilde oluşturmak anlamına gelir.

### 🔧 `pip` vs `poetry` vs `uv`

Bu üç araç da Python projelerinde paket ve bağımlılık yönetimi için kullanılır, ancak farklı yaklaşımları ve yetenekleri vardır:

---

### 🧩 `pip` (Python Package Installer)

* **Ne işe yarar?**
  Python’un resmi paket yöneticisidir. `PyPI` üzerindeki paketleri indirip yükler.

* **Özellikleri:**

  * En temel araçtır, her Python dağıtımında gelir.
  * `requirements.txt` ile çalışır.
  * Sadece paketleri kurar, sanal ortam yönetimi içermez.
  * Bağımlılık çözümlemesi sınırlıdır (çatışmaları düzgün çözemez).

* **Kullanım örneği:**

  ```bash
  pip install requests
  ```

---

### 📦 `poetry`

* **Ne işe yarar?**
  Hem paket kurulumu hem de proje yönetimi (versiyon, bağımlılık çözümü, yayınlama) yapar.

* **Özellikleri:**

  * `pyproject.toml` dosyası ile çalışır.
  * Sanal ortam oluşturur ve yönetir.
  * Bağımlılık çözümlemesi çok daha iyidir (`lock` dosyası üretir).
  * Proje versiyonlama ve yayınlama için de kullanılır.

* **Kullanım örneği:**

  ```bash
  poetry init
  poetry add numpy
  poetry install
  ```

---

### ⚡️ `uv` (Ultrafast Python Package Manager)

* **Ne işe yarar?**
  `pip` ve `poetry` gibi araçların yaptığı işleri **çok daha hızlı** yapmayı amaçlayan yeni nesil bir araçtır.

* **Özellikleri:**

  * `Rust` ile yazılmıştır → çok hızlıdır.
  * `pip`, `venv`, `poetry` ile uyumludur.
  * `requirements.txt` veya `pyproject.toml` fark etmeden çalışabilir.
  * Hem kurulum hem bağımlılık çözümlemesini yapar.

* **Kullanım örneği:**

  ```bash
  uv pip install pandas
  uv init my_project
  uv venv
  ```

---

Biz bu projede uv kuracağız. Bundan önce Rust kuruyoruz ki uv yi kurabilelim. Ardından gerekli dizine gelip Shell'de bunu çalıştırıyoruz:

**uv init claims_ml**

Sonrasında

**uv add numpy pandas**

Komut şu işlemleri yaptı:

* Proje dizininde `.venv` adlı sanal ortam oluşturuldu.
* `numpy` ve `pandas` kütüphaneleri yüklendi.
* Bu kütüphanelerin ihtiyaç duyduğu `python-dateutil`, `pytz`, `tzdata`, `six` gibi bağımlılıklar da otomatik olarak kuruldu.

Artık bu sanal ortamda `numpy` ve `pandas` hazır durumda. Ortamı aktive etmek için:

```
.venv\Scripts\activate
```

komutunu kullanabilirsin.


Wheel dosyaları (`.whl`), Python paketlerinin **önceden derlenmiş (binary)** formatta dağıtılmış halidir.

---

### Ne işe yarar?

* `.whl` uzantılı bu dosyalar, bir Python paketini kaynak koddan derlemek yerine **doğrudan hızlıca yüklemeye** yarar.
* Bu sayede `pip` gibi araçlar, örneğin `numpy` gibi derlemesi uzun süren paketleri saniyeler içinde kurabilir.

---

### Neden birden fazla wheel dosyası var?

Farklı işletim sistemleri ve Python sürümleri için farklı `.whl` dosyaları üretilir:

* `cp313-cp313`: Python 3.13 sürümü için.
* `macosx_10_13_x86_64` / `macosx_11_0_arm64`: İşletim sistemi ve mimariye göre.
* Windows, Linux ve Mac için farklı varyantlar olur.

---

### SHA256 hash ne işe yarar?

* `.whl` veya `.tar.gz` dosyasının **bütünlüğünü doğrulamak** için kullanılır.
* İndirdiğin dosya bu hash'e sahipse dosya bozulmamış ve orijinaldir.
* Güvenlik açısından önemlidir — dosyanın değiştirilmediğinden emin olunur.

---

Kısaca: `.whl` dosyaları paketi derlemeden hızlıca kurmak için kullanılır. Her sistem/sürüm için ayrı wheel olur. SHA256 ise dosyanın güvenliğini garanti eder.


* .lock dosyaları, projede kullanılan paketlerin sürüm bilgilerini sabitlemek için vardır.

* Amaç, projeyi biri başka bir bilgisayarda çalıştırdığında aynı paket sürümleriyle, aynı ortamın yeniden oluşturulmasını sağlamaktır.
Yani sadece "aynı paketler" değil, aynı sürümler de kurulsun diye.

## Sonuç

.venv sanal ortamı içinde numpy ve pandas gibi kütüphaneler yüklendi; bağımlılıkları da otomatik kuruldu. Wheel dosyaları bu paketlerin hızlı kurulumu için önceden derlenmiş sürümleridir ve sistem/sürüm uyumuna göre farklılık gösterir. .lock dosyası ise proje başka bir ortamda çalıştırıldığında aynı sürümlerin kurulmasını garanti eder.

Şimdi ortamımıza pytest kütüphanesi kuracağız fakat kullanıcı kişi için test gerekmediğinden dolayı şu şekilde dependency'yi ekliyoruz:

**uv add pytest --group dev**

* ruff format, Python dosyalarını otomatik olarak PEP8 uyumlu ve temiz bir şekilde biçimlendiren bir kod formatlama aracıdır.